#### Written by

David Yeo, Ph.D.

#### Introduction


The knapsack problem, which has been studied for more than a century, is an example of a combinatorial optimization problem. Its popularity is due to its wide-ranging applicability. As Technopedia put it,

>The knapsack problem can be found real-world scenarios like resource allocation in financial constraints or even in selecting investments and portfolios. It also can be found in fields such as applied mathematics, complexity theory, cryptography, combinatorics and computer science. It is easily the most important problem in logistics.

The following program uses q-learning to generate potential solutions to the knapsack problem. Specifically, it addresses the *unbounded knapsack problem*, which places no upper bound on the number of copies of each kind of item. More formally,

$$maximize \sum \limits _{i=1} ^{n} w_{i}v_{i}$$

$$where \sum \limits _{i=1} ^{n} w_{i}x_{i} \le W,  x_{i} \ge 0$$

Here $x_{i}$ represents the number of instances of item$_i$ to include in the knapsack, and $v_{i}$ and $w_{i}$ are (respectively) the value and the weight of the ith item. $W$ is the maximum weight the knapsack can carry. There are $n$ items that can potentially be packed. 

**Note:** An implicit constraint is $v_{i}, w_{i} > 0$. It makes no sense to pack items which have no value (i.e. where $v_{i} \le 0$).  And, although some items (like helium filled balloons) could have a  $w_{i} \le 0$, if the item has any value at all (i.e. if $v_{i} > 0$), the optimal strategy would be to pack an infinite number of those items. Remember the knapsack's capacity is defined by the total weight, not by volume (which is assumed to be infinite).

####  Program Listing

Two libraries are required: os and numpy.

In [1]:
import os
import numpy as np

As its name implies, the initialize_memory() function initializes the associative memory to the values stored in the specified long term memory file. If the memory file does not exist, a new associative memory is created and returned.

In [2]:
def initialize_memory(filename):
    if os.path.exists(filename): 
        memory = np.load(filename,allow_pickle='TRUE').item()
        return memory 
    return {}

Read_problem() loads the maximum knapsack capacity (W), the item value vector (v), and item weight (w) vector. These provide the constraints that define the problem to be solved.

In [3]:
def read_problem(filename):
    with open(filename) as fp:
        W = float(fp.readline())
        data = []
        for line in fp:
            data.append([float(x) for x in line.split()])
        v = np.array(data[0])
        w = np.array(data[1])
    return v,w,W

Get_possible() returns the list of indices for items that can validly be added to the knapsack. It first determines the amount of weight currently in the knapsack (current_total). It then iterates through each of the available items, checking if the addition of the item's weight would surpass the knapsack's capacity (W).

In [4]:
def get_possible(x,w,W):
    n = len(x)
    current_total = 0
    for i in range(n):
        current_total += w[i] * x[i]
    possible=[]
    for i in range(n): 
        if current_total + w[i] <= W:
            possible.append(i)
    return(np.array(possible,dtype=int))

The reinforcement() reward function is responsible for generating the r value in the q-learning equation. For the knapsack problem, this is given by the sum of the number of each item packed (${x_i}$) times the value of the item (${v_i}$).

In [5]:
def reinforcement(x,v):
    r = 0.0
    for i in range(len(x)):
        r += v[i] * x[i]
    return r

Get_q() retrieves the q-value of a specified state-action pair from the associative memory. If either the current state or state-action pair is unknown, get_q() initializes the q-value to a small uniform random value, and then stores it in the associative memory.

**Note:** According to reinforcement theory, any initial value works. The disadvantange of assigning a constant is that, when presented with a vector of constants (as occurs when training begins), the numpy.max() function favors the first occurrence. Since each q-value corresponds to an action, this means that the first action is preferentially selected. Initializing the q-values to small random values eliminates this bias.

In [6]:
def get_q(state,action):
    initial_value = np.random.uniform(high=0.1)
    actions = memory.get(str(state))
    if actions == None:
        memory[str(state)] = {str(action):initial_value}
        return initial_value
    q = actions.get(str(action))
    if q == None:
        memory[str(state)][str(action)] = initial_value
        return initial_value
    return q

Get_maxq() returns the maximum q-value for all of the actions that the puzzle has ever encountered. It takes advantage of the nested dictionary memory structure. The function begins by recalling all actions associated with the current state. If the state is unknown, a small uniform random value is returned. Otherwise the values() function recalls the q-values from all of the state's possible actions, and numpy.max() returns the largest.

In [7]:
def get_maxq(state):
    initial_value = np.random.uniform(high=0.1)
    actions = memory.get(str(state))
    if actions == None:
        return initial_value 
    values = list(actions.values())
    return np.max(values)

E_greedy() returns either a randomly selected action (exploration) or the action yielding the maximum q-value (exploitation). The specified epsilon value tells the e_greedy() function  how often, on average, to explore. The default is to explore 10% of the time. Setting epsilon = 0 forces e_greedy() to be greedy. 

The e_greedy() procedure starts by using the get_possible() procedure to generate a list of the items that can be added without violating the capacity limit. It then converts the possible values into a list of possible actions. If no action is possible, e_greedy() returns the value None. Otherwise, a small uniform random number is generated. If the random value is less than the epsilon parameter, one of the possible actions is randomly selected. Otherwise, the q-values of all of the current puzzle's possible actions are recalled from the associative memory, and the maximum of the q-values is returned.

In [8]:
def e_greedy(state,w,W,epsilon=0.1):
    actions = get_possible(state,w,W)
    nactions = len(actions)
    if nactions == 0:
        return None
    if np.random.uniform() < epsilon:
        action = actions[np.random.randint(0,nactions)]
    else:
        q_values = []
        for i in range(nactions):
            q_values.append(get_q(state,actions[i]))
        action = actions[np.argmax(q_values)]
    return action

The next procedure, apply_action(), applies the given action to a copy of the incoming state. The action is to add a specified amount (by default=1.0) to the item count in the state (i.e. x vector) identified by the action argument. In principle, any value can be used as the addend; it need not be a whole number. 

**Note:** This opens up the possibility of designing a function that changes the addend as the situation changes.

In [9]:
def apply_action(state,action,addend=1.0):
    successor = state.copy()
    successor[action] += addend
    return successor

The pack() function is another very important algorithm. It (greedily) packs the knapsack based on the action q-values recalled from memory. First, the current state (${s_1}$) of the knapsack is initialized to empty. Then an infinite loop begins. Because the epsilon parameter is set to zero, the greedy variant of e_greedy() is run. This recalls the action with the largest q-value. If no action is found, the infinite loop is broken and the current state (${s_1}$) is returned. Otherwise, the greedy action is applied to the current state, generating its successor (${s_2}$). The successor then becomes the current state in the next iteration.

In [10]:
def pack(w,W):
    s1 = np.zeros((len(w)))
    while True:
        action = e_greedy(s1,w,W,0)     # greedy solution (epsilon=0)
        if action == None:
            break
        print("add item",action+1,"to knapsack")
        s2 = apply_action(s1,action)
        s1 = s2
    return s1

The procedure responsible for devising advanced strategies is q_learn(). It is responsible for implementing the q-learning equation. 

Q_learn() begins by checking if the number of requested trials is > 0. If so, learning begins. First q_learn() determines the number of items to consider. It then executes the requested number of learning trials. At the beginning of each epoch the knapsack is emptied. It is refilled by entering into an infinite loop that can only be broken when e_greedy() determines that no action applies, i.e. the knapsack capacity has been reached. Otherwise, the current state's successor is generated by applying the e_greedy() selected action (${a_1}$) to the current state. This is the last component required to implement the q-learning equation:

$${q_1} = {q_1} + \alpha * (r + \gamma * {q_2} - c).$$

Get_q() recalls the current state's q-value (${q_1}$). Get_maxq() determines ${q_2}$, the largest q-value across all of the actions associated with the successor. And the reinforcement() function is tasked with estimating the successor's reward value (${r}$). The q-learning equation can now be applied. The updated ${q_1}$ value memorized, and the successor state becomes the current state on the next iteration of the infinite loop. 

Once training is complete, q_learn() calls the pack() procedure. Pack() greedily packs the knapsack and returns it as the proposed solution.

In [11]:
def q_learn(v,w,W,ntrials):
    if ntrials > 0:
        n = len(v)
        for t in range(ntrials):
            print("trial number: "+str(t+1), end="\r")
            s1 = np.zeros((n))  # empty knapsack
            while(True):
                a1 = e_greedy(s1,w,W)
                if a1 == None:
                    break
                s2 = apply_action(s1,a1)            
                r = reinforcement(s2,v)
                q1 = get_q(s1,a1)
                q2 = get_maxq(s2)
                q1 = q1 + alpha * (r + gamma * q2 - q1)
                memory[str(s1)][str(a1)] = q1
                s1 = s2
        print("trial number: "+str(t+1))
    print("\nProposed Solution\n")
    x = pack(w,W)
    return x

The print dictionary() function displays the contents of the long term memory.

In [12]:
def print_dictionary(memory):
    for key,value in memory.items():
        if isinstance(value,dict):
            items = dict(sorted(value.items()))
            actions = list(items.keys())
            q_values = list(items.values())
            best = np.argmax(q_values)
            print("\nx:",key)
            print("\nrecommended action:",actions[best],"\n")
            print_dictionary(items)
        else:
            print("action:",key,"\tq-value:",value)
    return

The main() procedure begins by setting the initial parameter values (e.g. learning rate). It then executes numpy's seed function. The seed function was only used to enable reproducibility of the pseudo-random results. It can be removed. Next the problem and memory files are identified. The associative memory is then recalled from the specified memory file. If the file does not exist, a new (tabula rasa) associative memory is created. 

**Note:** If the specified problem description file does not exist, Knapsack-Q simply exits.  

The problem to be solved is then passed to q_learn(), which generates and displays its potential solution. This is follows by a dump of the memory used to generate the proposed solution. Finally, the long term memory file is updated.

In [13]:
def main():

    global alpha                # learning rate (0:1]   
    global gamma                # discount factor (0:1] 
    global memory               # associative memory

    alpha = 0.25
    gamma = 0.95
    
    np.random.seed(19521212)    # can be removed
    
    data_file = "test.txt"
    memory_file = "test_ltm.npy" 

    if os.path.exists(data_file):
        v, w, W = read_problem(data_file)
        print("Input Data\n\nv:",v,"\nw:",w,"\n\nW:",W,"\n")
        memory = initialize_memory(memory_file)
        ntrials = int(input("trials requested: "))
        x = q_learn(v,w,W,ntrials)
        r = reinforcement(x,v)
        print("\nx:",x,"\tr:",r,"\n\nMemory Contents")
        print_dictionary(memory)
        np.save(memory_file,memory)

if __name__ == "__main__":
    main()

Input Data

v: [505. 352. 458. 220. 354. 414. 498. 545. 473. 543.] 
w: [23. 26. 20. 18. 32. 27. 29. 26. 30. 27.] 

W: 67.0 

trials requested: 1000
trial number: 1000

Proposed Solution

add item 1 to knapsack
add item 1 to knapsack
add item 3 to knapsack

x: [2. 0. 1. 0. 0. 0. 0. 0. 0. 0.] 	r: 1468.0 

Memory Contents

x: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

recommended action: 0 

action: 0 	q-value: 2789.4125142531893
action: 1 	q-value: 2258.0592855229015
action: 2 	q-value: 2475.8711740538606
action: 3 	q-value: 1963.1665190636563
action: 4 	q-value: 1013.4539940843529
action: 5 	q-value: 1271.4995151704538
action: 6 	q-value: 2216.5840482374942
action: 7 	q-value: 2634.8596529308697
action: 8 	q-value: 1955.3675602511753
action: 9 	q-value: 1553.5917272151103

x: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]

recommended action: 3 

action: 0 	q-value: 936.7395075758244
action: 1 	q-value: 790.199947467705
action: 2 	q-value: 930.1229063492727
action: 3 	q-value: 1560.3836287455993
action: 4 	q-va

##### Results

The problem illustrated was originally from the Wikipedia entry on the knapsack problem. The specified maximum capacity of the knapsack (W) is 67, and the value ($v_{i}$) and weight ($w_{i}$) of each item are:

* **v** = [505. 352. 458. 220. 354. 414. 498. 545. 473. 543.] 
* **w** = [23. 26. 20. 18. 32. 27. 29. 26. 30. 27.] 

After 1200 trials, the proposed solution (x) is [2. 0. 1. 0. 0. 0. 0. 0. 0. 0.]. First item 1 is added to the knapsack. Then another item 1 is packed. Finally, item 3 is added. This reaches the weight limit, i.e. 23 + 23 + 20 = 66. A brief history of the changing packing options as learning progresses is presented below.

     # Trials           Proposed Solution (x)          Total Value               

            0       [0. 0. 2. 0. 0. 0. 0. 0. 0. 1.]       1459.0       
            1       [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]        887.0
           10       [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]        887.0
          100       [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.] 	   887.0 
         1000       [0. 0. 0. 2. 0. 0. 0. 0. 1. 0.] 	   913.0
         1050       [0. 0. 0. 2. 0. 0. 0. 0. 1. 0.] 	   913.0
         1100       [0. 0. 0. 2. 0. 0. 0. 0. 1. 0.] 	   913.0
         1150       [0. 0. 0. 2. 0. 0. 0. 0. 1. 0.] 	   913.0
         1200       [2. 0. 1. 0. 0. 0. 0. 0. 0. 0.] 	  1468.0
         1250       [2. 0. 1. 0. 0. 0. 0. 0. 0. 0.] 	  1468.0
         1500       [2. 0. 1. 0. 0. 0. 0. 0. 0. 0.]       1468.0
         5000       [2. 0. 1. 0. 0. 0. 0. 0. 0. 0.] 	  1468.0
        10000       [2. 0. 1. 0. 0. 0. 0. 0. 0. 0.] 	  1468.0
        25000       [2. 0. 1. 0. 0. 0. 0. 0. 0. 0.] 	  1468.0

The maximum value of the packed knapsack seems to have stablized at 1468. Of course, it is possible that further training could discover an even more profitable solution. 

#### Discussion
     
Several revealing two item tests of the algorithm were run. These early explorations of the Knapsack-Q algorithm's ability are presented below. To increase the chance of convergence, in each of the following problems 25000 learning trials were requested. 

First, consider the case in which the more valueable item weighs less. There are two variants of this problem (see below). In both cases the Knapsack-Q algorithm found the optimal solution, i.e. select as many of the light weight, more valuable, items as possible.

    ----- Input Data -----            Proposed Solution       Value
      
    v=[1 2]  w=[2 1]  W=10                 [ 0. 10.] 	      20.0
    v=[2 1]  w=[1 2]  W=10                 [10.  0.] 	      20.0

Next consider the case were the weights are the same, but the values are different. The cases are itemized below. The optimal solution is clear; since all items weigh the same, pack as many of the most valuable items as possible. Again Knapsack-Q generated the optimal solution. 

    ----- Input Data -----            Proposed Solution       Value
    
    v=[1 2]  w=[1 1]  W=10                 [ 0. 10.] 	      20.0       
    v=[2 1]  w=[1 1]  W=10                 [10.  0.] 	      20.0              

Another interesting test is assigning the same value to both items. The two variants of this test are shown below. In both cases the optimal solution was discovered, i.e. take as many of the lighter weight items as possible.

    ----- Input Data -----            Proposed Solution       Value
    
    v=[1 1]  w=[1 2]  W=10                 [10.  0.] 	      10.0        
    v=[1 1]  w=[2 1]  W=10                 [ 0. 10.] 	      10.0

Finally, it is important to remember that neither the total capacity of the knapsack nor the value and the weight vectors need be whole numbers. The following problems illustrate this.

    ---------- Input Data ---------   Proposed Solution      Value
    
    v=[1.1 1.2]  w=[1.1 1.2]  W=10.5       [3. 6.]           10.5
    v=[1.2 1.1]  w=[1.1 1.2]  W=10.5       [9. 0.]           10.799999999999999
    v=[1.1 1.2]  w=[1.1 1.4]  W=10.5       [7. 2.]           10.100000000000001
    v=[1.1 1.2]  w=[1.3 1.4]  W=10.5       [0. 7.]            8.4    
    